In [1]:
from unsloth import FastLanguageModel
import torch

# 准备一些配置
max_seq_length = 2048  # 最大序列长度
dtype = None           # 让Unsloth自动检测
load_in_4bit = True    # 4bit量化，节省显存

# 让我们选择Qwen2.5-Coder-7B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./grpo_output/checkpoint-2100/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-15 20:19:11 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    NVIDIA vGPU-32GB. Num GPUs = 1. Max memory: 31.484 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [2]:
def get_answer(response: str):
    start_marker = '<|im_start|>assistant\n'
    end_marker = '<|im_end|>'
    # 查找开始和结束位置
    start_index = response.find(start_marker) + len(start_marker)
    end_index = response.find(end_marker, start_index)
    
    # 提取内容
    content = response[start_index:end_index]
    return content

In [3]:
def predict(prompt):
    # 使用微调模型进行预测
    FastLanguageModel.for_inference(model)  # 启用推理优化

    messages = [
        {"role": "user", "content": prompt}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=2048,
        temperature=1.5,
        use_cache=True
    )
    
    response = tokenizer.batch_decode(outputs)
    return get_answer(response[0])

In [4]:
import json
from openai import OpenAI
import os
from typing import List, Dict, Any
import csv

apikey = "sk-764f0662965b4ae3ad1cbe6067346100"
client = OpenAI(api_key=apikey, base_url="https://api.deepseek.com")
prompt_template = """请根据以下标准评估煤矿安全领域专家对特定学科问题的以下回答。您必须按照0、1、2或3颗星的评分标准对其进行评分：

总体评价：
0颗星表示错误答案和错误解释
1颗星表示回答错误，但部分解释合理
2颗星表示正确答案，部分合理解释
3星表示正确答案和合理解释

用户: {query}

LLM: {answer}

用户问题的正确答案是: {correct_answer}

您必须按照以下格式提供反馈: 
{"总体评分"：星级数（整数）}"""


def predict_with_ds(prompt: str):
    # 使用deepseek进行评估
    return client.chat.completions.create(
        model="deepseek-chat",
        messages=
        [
            {
                "role": "user",
                "content": prompt
            }
        ],
        top_p=0.7,
        temperature=0.1,
        stream=False,
        max_tokens=2500,
    ).choices[0].message.content


def eval(query: str, answer: str, correct_answer: str) -> int:
    _prompt = prompt_template.replace("{query}", query).replace("{answer}",answer).replace("{correct_answer}",correct_answer)
    print(_prompt)
    llm_response_str = predict_with_ds(_prompt)
    success = False
    retry_time = 0
    retry_max_time = 5
    eval_score = -1
    while not success and retry_time < retry_max_time:
        try:
            # 转化为字典
            eval_score = json.loads(llm_response_str)["总体评分"]
            success = True
        except:
            retry_time += 1
            continue
    print("eval_score: ",eval_score)
    return eval_score


def write_to_csv(data: List[Dict[str, Any]], filename: str, mode: str = 'w', encoding: str = 'utf-8-sig'):
    """
    公共的方法，将数据保存为csv文件

    :param data: 数据
    :param filename: 输出的文件名
    :param mode: 模式
    :param encoding: 编码
    :return:
    """
    if not data:
        print("警告：数据为空，未写入文件")
        return False
    try:
        fieldnames = set()
        fieldnames.update(data[0].keys())
        fieldnames = sorted(list(fieldnames))

        # 检查文件是否存在，以决定是否需要写入表头
        file_exists = os.path.isfile(filename)
        with open(filename, mode, newline='', encoding=encoding) as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            # 如果是新文件或者覆盖模式，写入表头
            if mode == 'w' or not file_exists:
                writer.writeheader()

            # 写入数据
            for item in data:
                # 确保每个数据项都包含所有字段，缺失字段用空字符串填充
                row = {field: item.get(field, '') for field in fieldnames}
                writer.writerow(row)
        print(f"数据已成功写入 {filename}")
        return True
    except Exception as e:
        print(f"写入CSV文件时出错: {e}")
        return False

In [5]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import jieba

# 确保已下载分词所需的数据
# nltk.download('punkt')


def calculate_bleu_scores(candidate, references):
    """
    计算BLEU 1-4gram分数和整体BLEU分数

    参数:
    candidate: 模型生成的文本
    references: 参考文本列表

    返回:
    包含各分数和平均值的字典
    """

    # 替换word_tokenize部分
    candidate_tokens = list(jieba.cut(candidate))
    reference_tokens = [list(jieba.cut(references))]

    # 初始化平滑函数
    smoothie = SmoothingFunction().method4

    # 计算各n-gram的分数
    scores = {}
    for n in range(1, 5):
        weights = [0] * 4
        weights[n - 1] = 1
        score = sentence_bleu(reference_tokens, candidate_tokens,
                              weights=weights,
                              smoothing_function=smoothie)
        scores[f'BLEU-{n}'] = score

    # 计算整体BLEU分数（使用标准权重）
    overall_bleu = sentence_bleu(reference_tokens, candidate_tokens,
                                 weights=(0.25, 0.25, 0.25, 0.25),
                                 smoothing_function=smoothie)
    scores['BLEU-Avg'] = overall_bleu

    return scores

In [6]:
from rouge import Rouge
import jieba

def cal_rouge_score(predict: str, ground_truth: str) -> dict:
    """
    计算rouge值

    :param predict: 模型预测
    :param ground_truth: 真实答案
    :return: rouge-1、rouge-2、rouge-l
    """
    hypothesis_seg = ' '.join(jieba.cut(predict))
    reference_seg = ' '.join(jieba.cut(ground_truth))
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis_seg, reference_seg)
    return {
        "rouge-1_r": scores[0]["rouge-1"]["r"],
        "rouge-1_p": scores[0]["rouge-1"]["p"],
        "rouge-1_f": scores[0]["rouge-1"]["f"],
        "rouge-2_r": scores[0]["rouge-2"]["r"],
        "rouge-2_p": scores[0]["rouge-2"]["p"],
        "rouge-2_f": scores[0]["rouge-2"]["f"],
        "rouge-l_r": scores[0]["rouge-l"]["r"],
        "rouge-l_p": scores[0]["rouge-l"]["p"],
        "rouge-l_f": scores[0]["rouge-l"]["f"],
    }

In [7]:
from tqdm import tqdm
with open("./test2.json", "r", encoding="utf-8") as fp: 
    data_list = json.load(fp)
llm_eval_score = 0
llm_eval_list = []

bleu_score = 0
bleu_list = []

rouge_f_score = 0
rouge_list = []

for data in tqdm(data_list):
    llm_response = predict(data["<input>"])
    
    rouge_value = cal_rouge_score(llm_response, data["<output>"])
    rouge_value["predict"] = llm_response
    rouge_value["ground_truth"] = data["<output>"]
    rouge_f_score += ((rouge_value["rouge-1_f"] + rouge_value["rouge-2_f"] + rouge_value["rouge-l_f"])/3)
    rouge_list.append(rouge_value)
    
    bleu_value = calculate_bleu_scores(llm_response, data["<output>"])
    bleu_value["predict"] = llm_response
    bleu_value["ground_truth"] = data["<output>"]
    bleu_list.append(bleu_value)
    bleu_score += bleu_value["BLEU-Avg"]

    
    llm_eval_value = eval(data["<input>"], llm_response,data["<output>"])
    llm_eval_list.append({"predict": llm_response, "ground_truth": data["<output>"], "llm_eval": llm_eval_value})
    llm_eval_score += llm_eval_value


  0%|          | 0/150 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.745 seconds.
DEBUG:jieba:Loading model cost 0.745 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


请根据以下标准评估煤矿安全领域专家对特定学科问题的以下回答。您必须按照0、1、2或3颗星的评分标准对其进行评分：

总体评价：
0颗星表示错误答案和错误解释
1颗星表示回答错误，但部分解释合理
2颗星表示正确答案，部分合理解释
3星表示正确答案和合理解释

用户: 煤矿建设安全规程的制定依据了哪些法律法规？

LLM: 煤矿建设安全规程的制定依据包括《煤炭法》、《矿山安全法》和《安全生产法》等法律，以及《煤矿安全规程》和《预防煤矿生产安全事故的条例》等行政法规。

用户问题的正确答案是: 煤矿建设安全规程是根据《煤炭法》《矿山安全法》《安全生产法》《职业病防治法》《煤矿安全监察条例》和《安全生产许可证条例》等法律法规制定的。

您必须按照以下格式提供反馈: 
{"总体评分"：星级数（整数）}


  1%|          | 1/150 [00:06<15:55,  6.41s/it]

eval_score:  2
请根据以下标准评估煤矿安全领域专家对特定学科问题的以下回答。您必须按照0、1、2或3颗星的评分标准对其进行评分：

总体评价：
0颗星表示错误答案和错误解释
1颗星表示回答错误，但部分解释合理
2颗星表示正确答案，部分合理解释
3星表示正确答案和合理解释

用户: 在中华人民共和国领域内从事煤炭生产和煤矿建设活动，必须遵守什么规程？

LLM: 在中华人民共和国领域内从事煤炭生产和煤矿建设活动，必须遵守《煤矿建设安全规程》。

用户问题的正确答案是: 在中华人民共和国领域内从事煤炭生产和煤矿建设活动，必须遵守煤矿建设安全规程。

您必须按照以下格式提供反馈: 
{"总体评分"：星级数（整数）}


  1%|▏         | 2/150 [00:10<12:37,  5.12s/it]

...

eval_score:  3
请根据以下标准评估煤矿安全领域专家对特定学科问题的以下回答。您必须按照0、1、2或3颗星的评分标准对其进行评分：

总体评价：
0颗星表示错误答案和错误解释
1颗星表示回答错误，但部分解释合理
2颗星表示正确答案，部分合理解释
3星表示正确答案和合理解释

用户: 煤矿企业投入生产前必须取得什么许可证？未取得该许可证的后果是什么？

LLM: 煤矿企业必须依据有关安全生产的法律、行政法规的规定取得安全生产许可证，未取得安全生产许可证的不得投入生产。

用户问题的正确答案是: 煤矿企业投入生产前必须依照有关安全生产的法律、行政法规的规定取得安全生产许可证，未取得安全生产许可证的不得从事煤炭生产。

您必须按照以下格式提供反馈: 
{"总体评分"：星级数（整数）}


 99%|█████████▉| 149/150 [14:39<00:05,  5.12s/it]

eval_score:  3
请根据以下标准评估煤矿安全领域专家对特定学科问题的以下回答。您必须按照0、1、2或3颗星的评分标准对其进行评分：

总体评价：
0颗星表示错误答案和错误解释
1颗星表示回答错误，但部分解释合理
2颗星表示正确答案，部分合理解释
3星表示正确答案和合理解释

用户: 对于特殊煤种或稀缺煤种，国家采取什么开采政策？

LLM: 对国民经济具有重要价值的特殊煤种或者稀缺煤种，国家实行保护性开采政策。

用户问题的正确答案是: 对国民经济具有重要价值的特殊煤种或者稀缺煤种，国家实行保护性开采。

您必须按照以下格式提供反馈: 
{"总体评分"：星级数（整数）}


100%|██████████| 150/150 [14:43<00:00,  5.89s/it]

eval_score:  3


In [8]:
write_to_csv(rouge_list, "./eval_result/gspo/rouge.csv")
write_to_csv(bleu_list, "./eval_result/gspo/bleu.csv")
write_to_csv(llm_eval_list, "./eval_result/gspo/llm_eval.csv")

写入CSV文件时出错: [Errno 2] No such file or directory: './eval_result/gspo/rouge.csv'
写入CSV文件时出错: [Errno 2] No such file or directory: './eval_result/gspo/bleu.csv'
写入CSV文件时出错: [Errno 2] No such file or directory: './eval_result/gspo/llm_eval.csv'


False

In [9]:
print("LLM-Eval:", llm_eval_score/(len(data_list)*3))
print("bleu_avg: ", bleu_score/len(data_list))
print("rouge:" , rouge_f_score/len(data_list))

LLM-Eval: 0.6333333333333333
bleu_avg:  0.3679473770615157
rouge: 0.5711766032579221


In [10]:
# 1400
# LLM-Eval: 0.6555555555555556
# bleu_avg:  0.3703089039968097
# rouge: 0.5742289516207081

# 1800
# LLM-Eval: 0.6133333333333333
# bleu_avg:  0.3493549138875762
# rouge: 0.5537011796547561